In [31]:
%%time
! python3 -m pip install --upgrade sagemaker
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role
role_name = role.split(["/"][-1])
print(f"SageMaker Execution Role:{role}")
print(f"The name of the Execution role: {role_name[-1]}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account:{account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region:{region}")

ClientError: An error occurred (ExpiredToken) when calling the GetCallerIdentity operation: The security token included in the request is expired

In [14]:
image = "efficientnet-smdataparallel-sagemaker"  # Example: mask-rcnn-smdataparallel-sagemaker
tag = "tf2.6"  # Example: pt1.8

In [7]:
!pygmentize ./Dockerfile

ARG region

FROM 763104351884.dkr.ecr.${region}.amazonaws.com/tensorflow-training:2.6.0-gpu-py38-cu112-ubuntu20.04

RUN pip install nvidia-pyindex nvidia-dllogger tensorflow-addons tensorflow-datasets

RUN pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110

RUN pip install pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-tf-plugin-cuda110


In [8]:
!pygmentize ./build_and_push.sh

#!/usr/bin/env bash
# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.
# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# set region

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"

if [ "$#" -eq 3 ]; then
    region=$1
    image=$2
    tag=$3
else
    echo "usage: $0 <aws-region> $1 <image-repo> $2 <image-tag>"
    exit 1
fi

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    echo "creating ECR repository : ${fullname} "
    aw

In [9]:
%%time
! chmod +x build_and_push.sh; bash build_and_push.sh {region} {image} {tag}

creating ECR repository : 570106654206.dkr.ecr.us-west-2.amazonaws.com/efficientnet-smdataparallel-sagemaker:efficientnet-smdataparallel-sagemaker-tf2.6 

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help

aws: error: argument operation: Invalid choice, valid choices are:

batch-check-layer-availability           | batch-delete-image                      
batch-get-image                          | complete-layer-upload                   
create-repository                        | delete-lifecycle-policy                 
delete-registry-policy                   | delete-repository                       
delete-repository-policy                 | describe-image-replication-status       
describe-image-scan-findings             | describe-images                         
describe-registry                        | describe-repositories                   
get-authorizat

In [18]:
# Training on SMDDP with S3 data source
import os
from sagemaker.tensorflow import TensorFlow

instance_type = "ml.p4d.24xlarge"
instance_count = 2
docker_image = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:{tag}"
username = "AWS"
job_name = "tf-smdataparallel-efficientnet"

In [24]:
# Configure the hyper-parameters
hyperparameters = {
    "mode" : "train",
    "arch": "efficientnet-b0"
}

In [26]:
estimator = TensorFlow(
    entry_point="main.py",
    role=role,
    image_uri=docker_image,
    source_dir="./smddp-adapt/efficientnet",
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="2.6",
    py_version="py38",
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters,
    debugger_hook_config=False,
    # Training using SMDataParallel Distributed Training Framework
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
)
estimator.fit(inputs='s3://smddp-570106654206-us-west-2/dataset/efficient/',job_name=job_name)

S3UploadFailedError: Failed to upload /var/folders/6s/bh1cyww95mg34p981v9cpjmmv5f05l/T/tmp8zydc4lz/source.tar.gz to sagemaker-us-west-2-570106654206/tf-smdataparallel-efficientnet/source/sourcedir.tar.gz: An error occurred (ExpiredToken) when calling the PutObject operation: The provided token has expired.

Horovod adaptation:
https://github.com/Arjunbala/DeepLearningExamples/tree/horovod-adapt

SMDDP adaptation:
https://github.com/Arjunbala/DeepLearningExamples/tree/smddp-adapt

